In [1]:
import duckdb
import os
import pyarrow.parquet as pq
import glob
import pandas as pd


## Load the all data with DuckDB ver1

In [2]:
os.chdir("..")
home_dir = os.getcwd()
# parquet_dir_node = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node")
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test")
duckdb_dir = os.path.join(home_dir, r"notebooks-contributions")
test_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test\ohsome_stats\type=way\state=history\year=2023")


In [3]:
home_dir = os.getcwd()
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test\ohsome_stats\type=way\state=history\year=2023", "")

# Construct the file pattern
file_pattern = os.path.join(parquet_dir, '*.parquet')
# print(file_pattern)

# Use glob to get the list of file names matching the pattern
parquet_files_dir = glob.glob(file_pattern)


In [4]:
parquet_files_dir


['c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\ohsome_stats\\type=way\\state=history\\year=2023\\data_0.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\ohsome_stats\\type=way\\state=history\\year=2023\\data_1.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\ohsome_stats\\type=way\\state=history\\year=2023\\data_10.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\oshdb-contributions-parquet-data-test\\ohsome_stats\\type=way\\state=history\\year=2023\\data_11.parquet',
 'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\

In [5]:
# Create a connection to the DuckDB database
con = duckdb.connect(database=f"{duckdb_dir}\\test.db", read_only=False)


In [6]:
# Step 5: Iterate over Parquet files and load into DuckDB
for file in parquet_files_dir:
    # Extract table name from file path
    table_name = file.split('/')[-1].split('.')[0].split("\\")[-1]
    table = pq.read_table(file)
    con.register(table_name, table)  # Register Parquet table in DuckDB


ArrowMemoryError: realloc of size 47185920 failed

In [7]:
table_name


'data_9'

In [8]:
parquet_files_dir[0].split('/')[-1].split('.')[0].split("\\")[-1]


'data_0'

In [8]:
# Step 6: Query the Parquet tables and load into DataFrame
dataframes = {}
for file in parquet_files_dir[0:1]:
    # Extract table name from file path
    table_name = file.split('/')[-1].split('.')[0].split("\\")[-1]
    result = con.execute(f"SELECT * FROM {table_name}")
    data = result.fetchall()
    columns = [desc[0] for desc in result.description]
    df = pd.DataFrame(data, columns=columns)
    dataframes[table_name] = df

# Example: Access the loaded dataframes
for table_name, df in dataframes.items():
    print(f"Data from {table_name}:")
    print(df)
    print()


Data from data_0:
         contrib_id  valid_from             valid_to     osm_id  osm_version  \
0            661377  1676448635           1680361279  234289724           16   
1            661642  1680362832           1680812423  234296718           13   
2            661661  1680362440           1680362832  234296718           12   
3            661678  1680362266           1680362440  234296718           11   
4            661828  1680362832           1680812423  234296720           11   
...             ...         ...                  ...        ...          ...   
1621726  2098190457  1674337438  9223372036854775807  511784412            3   
1621727  2098190486  1675910245           1675914135  706477274            3   
1621728  2098190584  1681301152           1682723014  724462445            3   
1621729  2098190641  1672660704           1674170494  531040692            4   
1621730  2098191562  1675612625  9223372036854775807   40556328            3   

         changeset_id

In [11]:
for root, dir_name, file_name in os.walk(parquet_dir):
    print(root)
    print(dir_name)
    print(os.basename(file_name))
    print("\n")

c:\Users\milan\OneDrive - MUNI\VŠ\PhD\Zahraniční stáž\Work\HeiGIT_notebooks\analysis\ai-assisted-osm-mapping-stats\oshdb-contributions-parquet-data-test
['ohsome_stats']


AttributeError: module 'os' has no attribute 'basename'

In [12]:
# Execute the SQL query
query = """
SELECT
  SUBSTRING(tags, start_pos + 1, end_pos - start_pos - 1) AS key_value_pair
FROM (
  SELECT
    tags,
    POSITION('=' IN tags) AS start_pos,
    POSITION('}' IN tags) AS end_pos
  FROM data_9
) subquery
WHERE key_value_pair LIKE '%maxar%';
"""
r1 = con.execute(query)

# Fetch and display the results
results = r1.fetchall()
for row in results:
    # if "maxar" in row:
    #     print(row)
    print(row)


('maxar, highway=track',)
('maxar, highway=primary, surface=asphalt, ref=RN 55, lanes=2, operator=وزارة الأشغال العمومية والنقل, smoothness=good, operator:wikidata=Q3073286, ford=yes, operator:wikipedia=ar:وزارة النقل (الجزائر), ref:ar=ط و 55, operator:ar=وزارة الأشغال العمومية والنقل',)
('maxar, highway=primary, surface=asphalt, ref=RN 55, lanes=2, operator=وزارة الأشغال العمومية والنقل, smoothness=good, construction=tertiary, operator:wikidata=Q3073286, ford=yes, operator:wikipedia=ar:وزارة النقل (الجزائر), ref:ar=ط و 55, operator:ar=وزارة الأشغال العمومية والنقل',)
('maxar, highway=track, tiger:county=Comanche, KS, tiger:cfcc=A41, tiger:reviewed=no',)
('maxar, highway=track, tiger:county=Comanche, KS, tiger:cfcc=A41, tiger:reviewed=no',)
('maxar, highway=primary, surface=asphalt, ref=RN 55, lanes=2, layer=1, smoothness=bad, ford=yes, ref:ar=ط و 55',)
('maxar, highway=unclassified, surface=unpaved',)
('maxar, highway=unclassified, surface=unpaved',)
('maxar, highway=unclassified, sur

In [17]:
# Execute the SQL query
query = """
WITH key_value_pairs AS (
    SELECT
        SUBSTRING(tags, start_pos + 1, end_pos - start_pos - 1) AS key_value_pair
    FROM (
        SELECT
            tags,
            POSITION('=' IN tags) AS start_pos,
            POSITION('}' IN tags) AS end_pos
        FROM data_9
    ) subquery
    WHERE key_value_pair LIKE '%maxar%'
)
SELECT
    ANY_VALUE(subquery.hashtags) AS "hashtags",
    ANY_VALUE(subquery.editor) AS "editor",
    ANY_VALUE(subquery.country) AS "country",
    subquery.user_id,
    COUNT(*) AS count
FROM (
    SELECT
        UNNEST(hashtags) AS hashtags,
        "country",
        "editor",
        "user_id"
    FROM data_9
    WHERE "hashtags" LIKE '%mapwithai%' AND "editor" LIKE '%RapiD%'
) AS subquery
WHERE EXISTS (
    SELECT 1
    FROM key_value_pairs
    WHERE subquery.hashtags LIKE '%' || key_value_pair || '%'
)
GROUP BY subquery.user_id;



"""
r1 = con.execute(query)

# Fetch and display the results
results = r1.fetchall()
for row in results:
    # if "maxar" in row:
    #     print(row)
    print(row)


: 

: 

In [10]:
# Execute the SQL query
query = """
    SELECT 
        ANY_VALUE(subquery.hashtags) AS "hashtags",
        ANY_VALUE(subquery.editor) AS "editor",
        ANY_VALUE(subquery.country) AS "country",
        ANY_VALUE(SUBSTRING(subquery.tags, start_pos + 1, end_pos - start_pos - 1)) AS key_value_pair,
        subquery.user_id,
        COUNT(*) AS count
    FROM (
        SELECT
            UNNEST(hashtags) AS "hashtags",
            "country",
            "editor",
            "user_id",
            "tags",
            POSITION('=' IN "tags") AS start_pos,
            POSITION('}' IN "tags") AS end_pos
        FROM data_0
        WHERE "hashtags" LIKE '%mapwithai%' AND "editor" LIKE '%RapiD%' --AND "tags" LIKE '%maxar%'
        GROUP BY "hashtags", "country", "editor", "user_id", "tags"
    ) AS subquery
    GROUP BY subquery.user_id



"""
r1 = con.execute(query)

# Fetch and display the results
results = r1.fetchall()
for row in results:
    # if "maxar" in row:
    #     print(row)
    print(row)


: 

: 

In [4]:
table = pq.read_table(test_dir)
con.register('test_table', table)


In [ ]:
results = con.execute('SELECT * FROM test_table').fetchall()


In [ ]:
len(results)

5629089

In [1]:
con.close()

NameError: name 'con' is not defined

## Load the all data with DuckDB ver2

In [1]:
import duckdb
import os
import pyarrow.parquet as pq
import glob
import pandas as pd


In [2]:
os.chdir("..")
home_dir = os.getcwd()
# parquet_dir_node = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node")
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test")
duckdb_dir = os.path.join(home_dir, r"notebooks-contributions")
test_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test\ohsome_stats\type=way\state=history\year=2023")


In [3]:
home_dir = os.getcwd()
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test\ohsome_stats\type=way\state=history\year=2023", "")

# Construct the file pattern
file_pattern = os.path.join(parquet_dir, '*.parquet')
# print(file_pattern)

# Use glob to get the list of file names matching the pattern
parquet_files_dir = glob.glob(file_pattern)


In [4]:
db = duckdb.read_parquet(parquet_files_dir)


In [5]:
db.columns

['contrib_id',
 'valid_from',
 'valid_to',
 'osm_id',
 'osm_version',
 'changeset_id',
 'changeset_timestamp',
 'hashtags',
 'editor',
 'user_id',
 'tags',
 'tags_delta',
 'primary_feature',
 'building',
 'road',
 'is_area',
 'area',
 'area_delta',
 'length',
 'length_delta',
 'contrib_type',
 'country_iso_a3',
 'country',
 'centroid',
 'geometry_type',
 'geometry_valid',
 'geometry',
 'state',
 'type']

In [6]:
r1 = duckdb.sql("""
    SELECT 
    
ANY_VALUE("country") AS "country", 
ANY_VALUE("editor") AS "editor",
"user_id", COUNT(*) AS count
 
    FROM db
    --WHERE "country" LIKE '%AN%'

    GROUP BY "user_id"
    ORDER BY count DESC
""")
r1.show()


┌─────────┬───────────────────────────────────────────┬──────────┬────────┐
│ country │                  editor                   │ user_id  │ count  │
│ varchar │                  varchar                  │  int32   │ int64  │
├─────────┼───────────────────────────────────────────┼──────────┼────────┤
│ PSE     │ osmtools/30252 (linux)                    │   145231 │ 573398 │
│ PSE     │ JOSM/1.5 (15390 en)                       │   494636 │ 509535 │
│ RUS     │ OSM Translator UA                         │  1538111 │ 211240 │
│ DEU     │ StreetComplete 52.0                       │   173981 │ 148042 │
│ UKR     │ reverter_plugin/36043;JOSM/1.5 (18678 ru) │ 10572603 │ 120489 │
│ DZA     │ JOSM/1.5 (18427 fr)                       │  5164102 │ 120379 │
│ BRA     │ JOSM/1.5 (18700 pt_BR)                    │ 15372998 │ 118421 │
│ USA     │ JOSM/1.5 (18570 en)                       │  8758904 │ 107349 │
│ UZB     │ JOSM/1.5 (18583 ru)                       │ 17925720 │ 100305 │
│ USA     │ 

In [7]:
r1 = duckdb.sql("""
    SELECT 
    
    UNNEST(hashtags) AS hashtags,
    "country"
 
    FROM db
    WHERE "hashtags" LIKE '%mapwithai%'

    --GROUP BY "country"
    
""")
r1.show()


┌──────────────┬─────────┐
│   hashtags   │ country │
│   varchar    │ varchar │
├──────────────┼─────────┤
│ #Tanzania    │ TZA     │
│ #mapwithai   │ TZA     │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
│ #Tanzania    │ KEN     │
│ #mapwithai   │ KEN     │
│ #Tanzania    │ KEN     │
│ #mapwithai   │ KEN     │
│ #Tanzania    │ KEN     │
│ #mapwithai   │ KEN     │
│   ·          │  ·      │
│   ·          │  ·      │
│   ·          │  ·      │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
│ #India       │ IND     │
│ #mapwithai   │ IND     │
├──────────────┴─────────┤
│ ? rows       2 columns │
└────────────────────────┘



In [8]:
r1 = duckdb.sql("""
    SELECT
            ANY_VALUE("country") AS "country",
            --ANY_VALUE("hashtags") AS "hashtags",
            ANY_VALUE("editor") AS "editor",
            "user_id",
            COUNT(*) AS count
        FROM db
        GROUP BY "user_id"
    
""")
r1.show()


┌─────────┬────────────────────────┬──────────┬───────┐
│ country │         editor         │ user_id  │ count │
│ varchar │        varchar         │  int32   │ int64 │
├─────────┼────────────────────────┼──────────┼───────┤
│ GBR     │ JOSM/1.5 (18646 en_GB) │  3951648 │ 22775 │
│ GBR     │ osmapi/3.0.0           │ 15274698 │  2194 │
│ AUT     │ JOSM/1.5 (18622 en)    │    29717 │  9909 │
│ GBR     │ iD 2.25.1              │   251688 │   145 │
│ BRA     │ JOSM/1.5 (18622 de)    │   629849 │   541 │
│ AUS     │ iD 2.24.2              │ 10036384 │   269 │
│ GRL     │ JOSM/1.5 (18646 en)    │   646021 │    33 │
│ USA     │ iD 2.25.1              │   123189 │   184 │
│ GBR     │ JOSM/1.5 (18621 en_GB) │   324405 │   146 │
│ NOR     │ JOSM/1.5 (18646 en_GB) │  3497659 │ 31700 │
│  ·      │     ·                  │     ·    │     · │
│  ·      │     ·                  │     ·    │     · │
│  ·      │     ·                  │     ·    │     · │
│ BOL     │ iD 2.23.2              │ 18242255 │ 

In [9]:
r1 = duckdb.sql("""
    SELECT
        tags
    FROM db;
""")
r1.show()


┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                         tags                                                         │
│                                                map(varchar, varchar)                                                 │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ {source=survey on 8th August 2007, highway=primary, name=King's Cross Road, surface=asphalt, oneway=yes, ref=A201,…  │
│ {highway=trunk, name=Gray's Inn Road, surface=asphalt, oneway=yes, ref=A5200, maxspeed=20 mph, lanes=4, operator=T…  │
│ {highway=trunk, name=Gray's Inn Road, oneway=yes, ref=A5200, maxspeed=30 mph, lanes=4, operator=Transport for Lond…  │
│ {highway=trunk, name=Gray's Inn Road, surface=asphalt, oneway=yes, ref=A5200, maxspeed=20 mph, lanes=4, operator=T…  │
│ {highway=trunk, name=Gray's In

In [12]:
r1 = duckdb.sql("""
SELECT
  SUBSTRING(subquery.tags, start_pos + 1, end_pos - start_pos - 1) AS key_value_pair
FROM (
  SELECT
    tags,
    POSITION('=' IN tags) AS start_pos,
    POSITION('}' IN tags) AS end_pos
  FROM db
) AS subquery
--WHERE key_value_pair LIKE '%maxar%';
""")
r1.show()


: 

: 

In [10]:
r1 = duckdb.sql("""
    SELECT
        SPLIT_PART(SPLIT_PART(tags, '=', 1), ',', 1) AS key1,
        SPLIT_PART(SPLIT_PART(tags, '=', 2), ',', 1) AS value1,
        SPLIT_PART(SPLIT_PART(tags, '=', 3), ',', 1) AS key2,
        SPLIT_PART(SPLIT_PART(tags, '=', 4), ',', 1) AS value2,
        -- Add more key-value pairs as needed
    FROM db;
""")
r1.show()


┌──────────┬───────────────────────────┬───────────────────┬───────────────────┐
│   key1   │          value1           │       key2        │      value2       │
│ varchar  │          varchar          │      varchar      │      varchar      │
├──────────┼───────────────────────────┼───────────────────┼───────────────────┤
│ {source  │ survey on 8th August 2007 │ primary           │ King's Cross Road │
│ {highway │ trunk                     │ Gray's Inn Road   │ asphalt           │
│ {highway │ trunk                     │ Gray's Inn Road   │ yes               │
│ {highway │ trunk                     │ Gray's Inn Road   │ asphalt           │
│ {highway │ trunk                     │ Gray's Inn Road   │ asphalt           │
│ {highway │ trunk                     │ Gray's Inn Road   │ yes               │
│ {highway │ trunk                     │ Gray's Inn Road   │ asphalt           │
│ {highway │ trunk                     │ Euston Road       │ asphalt           │
│ {highway │ trunk          

In [11]:
r1 = duckdb.sql("""
SELECT
  SPLIT_PART(subquery.kv, '=', 1) AS tag_key,
  SPLIT_PART(subquery.kv, '=', 2) AS tag_value
FROM (
  SELECT
    UNNEST(STRING_TO_ARRAY(REPLACE(REPLACE(tags, '{', ''), '}', ''), ',')) AS kv
  FROM db
) AS subquery
--WHERE tag_key LIKE '%maxar%' OR tag_value LIKE '%maxar%';
""")

r1.show()


┌───────────┬───────────────────────────┐
│  tag_key  │         tag_value         │
│  varchar  │          varchar          │
├───────────┼───────────────────────────┤
│ source    │ survey on 8th August 2007 │
│  highway  │ primary                   │
│  name     │ King's Cross Road         │
│  surface  │ asphalt                   │
│  oneway   │ yes                       │
│  ref      │ A201                      │
│  maxspeed │ 20 mph                    │
│  lanes    │ 2                         │
│  lit      │ yes                       │
│  sidewalk │ both                      │
│    ·      │  ·                        │
│    ·      │  ·                        │
│    ·      │  ·                        │
│ highway   │ residential               │
│  name     │ دانش آموز 13              │
│  surface  │ asphalt                   │
│  maxspeed │ 30                        │
│ highway   │ residential               │
│  name     │ دانش آموز14               │
│  surface  │ asphalt             

In [12]:
r1 = duckdb.sql("""
SELECT
  --ANY_VALUE("country") AS "country",
  SPLIT_PART(subquery.kv, '=', 1) AS tag_key,
  SPLIT_PART(subquery.kv, '=', 2) AS tag_value

FROM (
  SELECT
  UNNEST(STRING_TO_ARRAY(REPLACE(REPLACE(tags, '{', ''), '}', ''), ',')) AS kv
  from db
  ) AS subquery
--WHERE tag_key == 'maxspeed';
""")

r1.show()


┌───────────┬───────────────────────────┐
│  tag_key  │         tag_value         │
│  varchar  │          varchar          │
├───────────┼───────────────────────────┤
│ source    │ survey on 8th August 2007 │
│  highway  │ primary                   │
│  name     │ King's Cross Road         │
│  surface  │ asphalt                   │
│  oneway   │ yes                       │
│  ref      │ A201                      │
│  maxspeed │ 20 mph                    │
│  lanes    │ 2                         │
│  lit      │ yes                       │
│  sidewalk │ both                      │
│    ·      │  ·                        │
│    ·      │  ·                        │
│    ·      │  ·                        │
│ highway   │ residential               │
│  name     │ دانش آموز 13              │
│  surface  │ asphalt                   │
│  maxspeed │ 30                        │
│ highway   │ residential               │
│  name     │ دانش آموز14               │
│  surface  │ asphalt             

In [33]:
r1 = duckdb.sql("""
    SELECT
      JSON_EXTRACT(tags, '$."highway"') AS key1
    FROM db;
""")
r1.show()


InvalidInputException: Invalid Input Error: Malformed JSON at byte 1 of input: unexpected character.  Input: {highway=track, access=private, tracktype=grade2}

In [9]:
r1 = duckdb.sql("""
    SELECT 
        ANY_VALUE(subquery.hashtags) AS "hashtags",
        ANY_VALUE(subquery.editor) AS "editor",
        ANY_VALUE(subquery.country) AS "country",
        ANY_VALUE(SUBSTRING(subquery.tags, start_pos + 1, end_pos - start_pos - 1)) AS key_value_pair,
        subquery.user_id,
        COUNT(*) AS count
        
    FROM (
        SELECT
            UNNEST(hashtags) AS "hashtags",
            "country",
            "editor",
            "user_id",
            "tags",
            POSITION('=' IN tags) AS start_pos,
            POSITION('}' IN tags) AS end_pos
 
        FROM db
        WHERE "hashtags" LIKE '%mapwithai%' AND "editor" LIKE '%RapiD%' --AND "tags" LIKE '%maxar%'
        GROUP BY hashtags, "country", "editor", "user_id"
        
    ) AS subquery

    GROUP BY subquery.user_id
""")
r1.show()


BinderException: Binder Error: column "tags" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(tags)" if the exact value of "tags" is not important.

In [16]:
r1 = duckdb.sql("""
    SELECT 
        ANY_VALUE(subquery.hashtags) AS "hashtags",
        ANY_VALUE(subquery.editor) AS "editor",
        ANY_VALUE(subquery.country) AS "country",
        ANY_VALUE(subquery.tags) AS "tags",
        subquery.user_id,
        COUNT(*) AS count
        
    FROM (
        SELECT
            UNNEST(hashtags) AS "hashtags",
            "country",
            "editor",
            "user_id",
            ARRAY_AGG(DISTINCT tags) AS "tags"
 
        FROM db
        WHERE "hashtags" LIKE '%mapwithai%' AND "editor" LIKE '%RapiD%' AND "tags" LIKE '%maxar%'
        GROUP BY hashtags, "country", "editor", "user_id"
        
    ) AS subquery

    GROUP BY subquery.user_id
""")
r1.show()


: 

: 

In [ ]:
from (
    select date, cast(json_parse(volume_info) as map(varchar, varchar)) m
    from dataset
)
cross join unnest(map_keys(m), map_values(m)) as t(name, value)


In [13]:
r1 = duckdb.sql("""
    SELECT 
        ANY_VALUE(subquery.hashtags) AS "hashtags",
        ANY_VALUE(subquery.editor) AS "editor",
        ANY_VALUE(subquery.country) AS "country",
    
        subquery.user_id,
        COUNT(*) AS count
        
    FROM (
        SELECT
            UNNEST(hashtags) AS hashtags,
            "country",
            "editor",
            "user_id",
            
 
            FROM db
            WHERE "hashtags" LIKE '%mapwithai%' AND "editor" LIKE '%RapiD%'
        
    ) AS subquery

    GROUP BY subquery.user_id
    
""")
r1.show()


┌──────────────────┬──────────────────┬─────────┬──────────┬───────┐
│     hashtags     │      editor      │ country │ user_id  │ count │
│     varchar      │     varchar      │ varchar │  int32   │ int64 │
├──────────────────┼──────────────────┼─────────┼──────────┼───────┤
│ #Tanzania        │ RapiD 1.1.9-tm.1 │ TZA     │  9431905 │  2378 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979431 │  4206 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979445 │  4782 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979427 │  3436 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │  9124220 │  2276 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979441 │  1614 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979432 │  1748 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │  9124217 │  3388 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10979444 │  4236 │
│ #India           │ RapiD 1.1.9-tm.1 │ IND     │ 10797439 │  6294 │
│   ·              │      ·       

In [9]:
r1 = duckdb.sql("""
    SELECT 
        ANY_VALUE(subquery.hashtags) AS "hashtags",
        ANY_VALUE(subquery.editor) AS "editor",
        ANY_VALUE(subquery.country) AS "country",
    
        subquery.user_id,
        COUNT(*) AS count
        
    FROM (
        SELECT
            UNNEST(hashtags) AS hashtags,
            "country",
            "editor",
            "user_id",
            "tags"::string
            
 
            FROM db
            WHERE "tags" LIKE '%RapiD%'
        
    ) AS subquery

    GROUP BY subquery.user_id
    
""")
r1.show()


: 

: 

In [10]:
r1 = duckdb.sql("""
SELECT 
    ANY_VALUE(subquery.hashtags) AS hashtags,
    ANY_VALUE(subquery.editor) AS editor,
    ANY_VALUE(subquery.country) AS country,
    subquery.user_id,
    COUNT(*) AS count
FROM (
    SELECT
        UNNEST(hashtags) AS hashtags,
        country,
        editor,
        user_id,
        tags::string
    FROM db
    WHERE tags LIKE '%RapiD%'
) AS subquery
GROUP BY subquery.user_id;
    
""")
r1.show()


: 

: 

In [34]:
r1 = duckdb.sql("""
    SELECT 
    
    "contrib_id", "osm_id", "hashtags", "editor", "user_id", "tags"--, "type"
/*
    "length", "contrib_type", "country_iso_a3", "country",

    "geometry_type", "state"
 */
    FROM db

    LIMIT 10;
""")
r1.show()


┌────────────┬───────────┬──────────────┬──────────────────────┬─────────┬─────────────────────────────────────────────┐
│ contrib_id │  osm_id   │   hashtags   │        editor        │ user_id │                    tags                     │
│   int64    │   int64   │  varchar[]   │       varchar        │  int32  │            map(varchar, varchar)            │
├────────────┼───────────┼──────────────┼──────────────────────┼─────────┼─────────────────────────────────────────────┤
│     661377 │ 234289724 │ []           │ JOSM/1.5 (18646 en…  │ 3951648 │ {source=survey on 8th August 2007, highwa…  │
│     661642 │ 234296718 │ []           │ JOSM/1.5 (18678 en…  │ 3951648 │ {highway=trunk, name=Gray's Inn Road, sur…  │
│     661661 │ 234296718 │ [#134385200] │ reverter_plugin/36…  │ 3951648 │ {highway=trunk, name=Gray's Inn Road, one…  │
│     661678 │ 234296718 │ []           │ JOSM/1.5 (18678 en…  │ 3951648 │ {highway=trunk, name=Gray's Inn Road, sur…  │
│     661828 │ 234296720 │ []   

In [37]:
duckdb.sql("""

""")


ParserException: Parser Error: syntax error at or near "}"
LINE 3: ...- use a list of globs to read all parquet files from 2 specific folders
SELECT * FROM read_parquet({parquet_files_dir});
                                                  ^

In [38]:
r1 = duckdb.sql("""
    SELECT table_name, column_name
    FROM information_schema.columns;
""")
r1.show()


┌────────────┬─────────────┐
│ table_name │ column_name │
│  varchar   │   varchar   │
├──────────────────────────┤
│          0 rows          │
└──────────────────────────┘



In [24]:
r1 = duckdb.sql("""
    DESCRIBE system.information_schema.tables
""")
r1.show()


┌──────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│         column_name          │ column_type │  null   │   key   │ default │ extra │
│           varchar            │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ table_catalog                │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ table_schema                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ table_name                   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ table_type                   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ self_referencing_column_name │ NULL        │ YES     │ NULL    │ NULL    │  NULL │
│ reference_generation         │ NULL        │ YES     │ NULL    │ NULL    │  NULL │
│ user_defined_type_catalog    │ NULL        │ YES     │ NULL    │ NULL    │  NULL │
│ user_defined_type_schema     │ NULL        │ YES     │ NULL    

In [19]:

r1 = duckdb.sql("""
    SELECT
        count(*) as n_edits
        ,count(distinct user_id) as n_users
    FROM db
    WHERE editor ^@ 'Street'
""")
r1.show()

┌─────────┬─────────┐
│ n_edits │ n_users │
│  int64  │  int64  │
├─────────┼─────────┤
│ 1922390 │   11734 │
└─────────┴─────────┘



In [6]:
con.close()